# Erste Schritte

Zunächst feststellen wo wir sind:

In [1]:
pwd

/home/vagrant/rdf/ais/hands-on-ansible/00-home-base


Und welche Dateien es dort gibt:

In [2]:
ls -lachi

insgesamt 92K
1580296 drwxrwxr-x 3 vagrant vagrant 4,0K Jan 17 07:49 .
1580253 drwxrwxr-x 8 vagrant vagrant 4,0K Nov 13 23:09 ..
1580297 -rw-rw-r-- 1 vagrant vagrant   60 Nov 13 23:09 2.1_creating_Ansible_Home_Base.sh
1580298 -rw-rw-r-- 1 vagrant vagrant  359 Nov 13 23:09 2.2_Ansible_Bleeding_Edge.sh
1580299 -rw-rw-r-- 1 vagrant vagrant  498 Nov 13 23:09 2.3_Ansible_pip.sh
1580300 -rwxrwxr-x 1 vagrant vagrant  737 Nov 13 23:09 2.4_lxc_setup.sh
1580302 -rwxrwxr-x 1 vagrant vagrant 1,2K Nov 13 23:09 ais-lxc-setup.sh
1580896 -rw-r--r-- 1 vagrant vagrant  19K Jan 17 07:37 ansible.cfg
1580301 -rw-rw-r-- 1 vagrant vagrant  25K Jan 17 07:02 Erste_Schritte_LXC.ipynb
1581120 -rw-rw-r-- 1 vagrant vagrant   41 Jan 17 07:45 hosts
1580858 drwxr-xr-x 2 vagrant vagrant 4,0K Jan 17 07:00 .ipynb_checkpoints
1580894 -rw-r--r-- 1 vagrant vagrant   10 Jan 17 07:01 setup-lxc.retry
1580303 -rw-rw-r-- 1 vagrant vagrant  372 Nov 13 23:09 setup-lxc.yml


Der Inhalt der Datei ais-lxc-setup.sh

In [3]:
cat ais-lxc-setup.sh

#!/bin/bash

set -uxe

# Path for prepare playbook
PREPARE=~/rdf/ais/hands-on-ansible/02-playbooks/00-simple-playbook-examples/prepare_ansible_target.yml
# Create ssh-key with empty passphrase (ONLY FOR TESTING)
MAKE_SSH_KEY="ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N \"\""

# Create containers
# The login user is ubuntu with password ubuntu
for i in ais-bashy web1 web2 db playbooks; do
	sudo lxc-create -n ${i} -t ubuntu && \
	# Remove legacy configuration keys \
	sudo lxc-update-config -c /var/lib/lxc/${i}/config && \
	# Start container \ 
	sudo lxc-start -n ${i} && \
	# Prepare ais-bashy \
	if [[ ${i} == "ais-bashy" ]]; then
		echo "Wait for comming up"
		sleep 5
		# Get ip
		ip=$(sudo lxc-ls --fancy | grep ${i} | cut -d '-' -f 3 | tr -d ' ')
		# Remove host key for ip
		ssh-keygen -R ${ip}
		# Add ip into known_hosts
		ssh-keyscan -H ${ip} >> ~/.ssh/known_hosts
		# Setup ssh-key for user
		ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a "${MAKE_SSH_KEY}" --connection=loc

Dieses Skript ist interaktiv, es frägt nach dem Loginpasswort `ubuntu` und suod-Passwort `ubuntu`.

Mit `expect` bzw. mit dem python-Modul pexpect können diese Fragen per Skript beantwortet werden.

https://www.clug.de/vortraege/expect/expect.html

Für `expect` gibt es das gleichnamige ansible-Modul. Die Dokumentation findet man hier:

http://docs.ansible.com/ansible/latest/expect_module.html#requirements-on-host-that-executes-module

Es werden die Abhängigkeiten installiert:

In [4]:
sudo apt-get update && sudo apt-get install --yes python-pexpect python3-pexpect

Ign:1 http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 InRelease
Holen:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]   
OK:3 http://ppa.launchpad.net/ansible/ansible/ubuntu xenial InRelease          
Ign:4 http://dl.google.com/linux/chrome/deb stable InRelease                   
OK:5 http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 Release         
OK:6 http://dl.google.com/linux/chrome/deb stable Release                      
OK:7 http://ppa.launchpad.net/git-core/ppa/ubuntu xenial InRelease             
OK:8 http://us.archive.ubuntu.com/ubuntu xenial InRelease                      
Fehl:9 http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 Release.gpg   
  Die folgenden Signaturen waren ungültig: KEYEXPIRED 1515625755
OK:10 http://ppa.launchpad.net/gnome-terminator/ppa/ubuntu xenial InRelease    
OK:13 https://download.docker.com/linux/ubuntu xenial InRelease                
OK:14 https://deb.nodesource.com/node_6.x xenial InRelease   

Somit kann das ansible-Module `expect` verwendet werden.

In [5]:
cat setup-lxc.yml

---
- hosts: all
  connection: local
  gather_facts: no

  tasks:
  - name: Update apt
    apt:
      update_cache: yes
      cache_valid_time: 3600
      
  - name: Init AIS lxc containers
    expect:
      command: /home/vagrant/rdf/ais/hands-on-ansible/00-home-base/ais-lxc-setup.sh
      responses:
        (?i)password: "ubuntu"
      creates: /var/lib/lxc/ais-bashy


Unser Skript wird aufgerufen und falls der String `password` (case insensitive) erscheint, wird `ubuntu` als Passwort eingegeben.

In [6]:
ansible-playbook -i localhost, setup-lxc.yml

 ____________
< PLAY [all] >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

 ___________________
< TASK [Update apt] >
 -------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

ok: [localhost]
 ________________________________
< TASK [Init AIS lxc containers] >
 --------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

changed: [localhost]
 ____________
< PLAY RECAP >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

localhost                  : ok=2    changed=1    unreachable=0    failed=0   



Wir können nun die Container anzeigen lassen.

In [7]:
sudo lxc-ls

The configuration file contains legacy configuration keys.
Please update your configuration file!
ais-bashy  db         playbooks  user-bashy web1       web2       


In [8]:
sudo lxc-ls --help

Usage: lxc-ls 
[-P lxcpath] [--active] [--running] [--frozen] [--stopped] [--nesting] [-g groups] [--filter regex]
[-1] [-P lxcpath] [--active] [--running] [--frozen] [--stopped] [--nesting] [-g groups] [--filter regex]
[-f] [-P lxcpath] [--active] [--running] [--frozen] [--stopped] [--nesting] [-g groups] [--filter regex]

lxc-ls list containers

Options :
  -1, --line         show one entry per line
  -f, --fancy        use a fancy, column-based output
  -F, --fancy-format comma separated list of columns to show in the fancy output
                     valid columns are: NAME, STATE, PID, RAM, SWAP, AUTOSTART,
                     GROUPS, INTERFACE, IPV4 and IPV6
  --active           list only active containers
  --running          list only running containers
  --frozen           list only frozen containers
  --stopped          list only stopped containers
  --defined          list only defined containers
  --nesting=NUM      list nested containers up to NUM (default is 5) levels of

In [9]:
sudo lxc-ls --fancy

The configuration file contains legacy configuration keys.
Please update your configuration file!
NAME       STATE   AUTOSTART GROUPS IPV4       IPV6 
ais-bashy  RUNNING 0         -      10.0.3.85  -    
db         RUNNING 0         -      10.0.3.84  -    
playbooks  RUNNING 0         -      10.0.3.103 -    
user-bashy STOPPED 0         -      -          -    
web1       RUNNING 0         -      10.0.3.197 -    
web2       RUNNING 0         -      10.0.3.70  -    


Nur für unseren ais-bashy lautet das Kommando:

In [10]:
sudo lxc-ls ais-bashy --fancy

NAME      STATE   AUTOSTART GROUPS IPV4      IPV6 
ais-bashy RUNNING 0         -      10.0.3.85 -    


In [11]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $1 }'

NAME
ais-bashy


In [12]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $2 }'

STATE
RUNNING


In [13]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $3 }'

AUTOSTART
0


In [14]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $4 }'

GROUPS
-


In [15]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }'

IPV4
10.0.3.85


In [16]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }' | grep '.\.'

10.0.3.85


In [17]:
export AIS_BASHY_IP=$(sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }' | grep '.\.')
echo $AIS_BASHY_IP

10.0.3.85


## Uptime via ssh und ansible erfragen

In [18]:
ssh root@${AIS_BASHY_IP} uptime

 07:51:35 up 55 min,  0 users,  load average: 2,39, 0,91, 0,48


In [19]:
ls ~/.ssh

authorized_keys  id_rsa  id_rsa.pub  known_hosts


In [20]:
type echo

echo ist eine von der Shell mitgelieferte Funktion.


In [21]:
help echo

echo: echo [-neE] [Argument ...]
    Ausgabe der Argumente auf die Standardausgabe.
    
    Zeigt die Argumente auf der Standardausgabe gefolgt von einem
    Zeilenumbruch an.
    
    Optionen:
      -n	Keinen Zeilenumbruch anfügen
      -e	Interpretation der folgenden Escape-Sequenzen zulassen
      -E	Keine Interpretation der Escape-Sequenzen.
    
    »echo« interpretiert die folgenden Escape-Sequenzen:
      \a	Alarm (Glocke)
      \b	Rücktaste (Backspace)
      \c	weitere Ausgabe unterdrücken
      \e	Escape-Zeichen
      \E        Escape Zeichen
      \f	Seitenvorschub
      \n	Zeilenvorschub
      \r	Wagenrücklauf
      \t	Horizontaler Tabulator
      \v	Vertikaler Tabulator
      \\        umgekehrter Schrägstrich (Backslash)
      \0nnn	Zeichen mit dem ASCII-Code »NNN« (oktal). »NNN« kann null
    		bis drei oktale Ziffern haben.
      \xHH	Acht-Bit-Zeichen mit dem Wert »HH« (hexadezimal). »HH«
    		kann ein oder zwei hexadezimale Ziffern haben.
    
    Rückgabewert:
    G

Achtung dies überschreibt eine bestehende ssh-config Datei.

In [22]:
echo -e "HOST AIS-BASHY\n\tHostName\t${AIS_BASHY_IP}\n\tUser\troot" > ~/.ssh/config

In [23]:
cat ~/.ssh/config

HOST AIS-BASHY
	HostName	10.0.3.85
	User	root


In [24]:
ssh AIS-BASHY uptime

 07:51:36 up 55 min,  0 users,  load average: 2,19, 0,89, 0,47


In [25]:
ansible -a uptime -i ${AIS_BASHY_IP}, ${AIS_BASHY_IP} -u root

10.0.3.85 | SUCCESS | rc=0 >>
 07:51:39 up 55 min,  1 user,  load average: 2,19, 0,89, 0,47


## Aufräumen

Achtung dies löscht alle Container ohne wenn und aber.

In [ ]:
for i in ais-bashy db web1 web2 playbooks; do
    sudo lxc-destroy --force --name ${i}
done

In [ ]:
sudo lxc-ls --fancy